In [ ]:


def findHomology(sequence1, sequence2): #Returns largest homologous region between the 2 sequences
    m = [[0] * (1+len(sequence2))
    for i in range(1+len(sequence1))]
        longest, x_longest = 0, 0
        for x in range(1, 1 + len(sequence2)):
            for y in range(1, 1 + len(sequence2)):
                if sequence1[x - 1] == sequence2[y - 1]:
                    m[x][y] = m[x - 1][y - 1] + 1
                    if m[x][y] > longest:
                        longest = m[x][y]
                        x_longest = x
                else:
                    m[x][y] = 0
    return sequence1[x_longest - longest: x_longest]
        
def choosePrimers(insert, upstreamBackbone, downstreamBackbone): #Make primers for an insert to amplify it such that it can fit in a 
    forwardPrimer = insert[:20]
    reversePrimer = reverseComplement(insert[len(insert) - 20: len(insert)])
    getTm
    
def reverseComplement(sequence):
    reverseComplement = ""
    for base in sequence:
        if base == "a":
            reverseComplement = reverseComplement + "t"
        elif base == "t":
            reverseComplement = reverseComplement + "a"
        elif base == "g":
            reverseComplement = reverseComplement + "c"
        elif base == "c":
            reverseComplement = reverseComplement + "g"
        else:
            print ("error")
    return (reverseComplement)
def getTm(sequence):
    composition = getComposititon(sequence)
    #Tm= 64.9 +41*(yG+zC-16.4)/(wA+xT+yG+zC) 
    Tm = 64.9 + 41*(composition["G"] + composition["C"] - 16.4) / (composition["A"] + composition["T"] + composition["G"] + composition["C"])
    return (Tm)
def getComposition(sequence):
    composition = {"A" : 0, "G" : 0, "C" : 0, "T" : 0}
    for base in sequence:
        if base == "a":
            composition["A"] = composition["A"] + 1
        elif base == "t":
            composition["T"] = composition["T"] + 1
        elif base == "g":
            composition["G"] = composition["G"] + 1
        elif base == "c":
            composition["C"] = composition["C"] + 1
        else:
            print("error 2")
    return composititon

In [120]:
import primer3

class DNASequence():
    def __init__(self, sequence):
        #Initialize the DNA variable, which has sequence info as well as additional properties (Tm, homodimer & hairpin formation)
        self.sequence = sequence
        self.composition = self.getComposition(sequence)
        if len(sequence) != 0:
            self.gcContent = self.getGCcontent(sequence)
        else:
            self.gcContent = 0
        if len(sequence) < 60:
            self.Tm = primer3.calcTm(sequence)
            self.Homodimer = primer3.calcHomodimer(sequence)
            self.Hairpin = primer3.calcHairpin(sequence)
        else:
            self.Tm = 100000 #set the Tm arbitrarily high
        
    def getComposition(self, sequence):
        composition = {"A" : 0, "G" : 0, "C" : 0, "T" : 0}
        for base in sequence:
            if base == "A":
                composition["A"] = composition["A"] + 1
            elif base == "T":
                composition["T"] = composition["T"] + 1
            elif base == "G":
                composition["G"] = composition["G"] + 1
            elif base == "C":
                composition["C"] = composition["C"] + 1
            else:
                print("error 2")
        return composition
    def getGCcontent(self, sequence):
        GCpercent = (self.composition["G"] + self.composition["C"])/(self.composition["G"] + self.composition["T"] + self.composition["A"] + self.composition["C"])
        return GCpercent
    def reverseComplement(self, sequence):
        #Takes the complement of the sequence and reverses it.
        reverseComplement = ""
        for base in sequence:
            if base == "A":
                reverseComplement = "T" + reverseComplement
            elif base == "T":
                reverseComplement = "A" + reverseComplement
            elif base == "G":
                reverseComplement = "C" + reverseComplement
            elif base == "C":
                reverseComplement = "G" + reverseComplement 
            else:
                print ("error")
        return (reverseComplement)
def reverseComplement(sequence):
    #Takes the complement of the sequence and reverses it.
    reverseComplement = ""
    for base in sequence:
        if base == "A":
            reverseComplement = "T" + reverseComplement 
        elif base == "T":
            reverseComplement = "A" + reverseComplement
        elif base == "G":
            reverseComplement = "C" + reverseComplement
        elif base == "C":
            reverseComplement = "G" + reverseComplement
        else:
            print ("error")
    return (reverseComplement)
def screenPrimers(insert, upstreamBackbone, downstreamBackbone):
    #run the main program to create and screen the forward and reverse primers and then add overhangs.
    forwardPrimer = screenPrimer(insert, "F")
    print(forwardPrimer.sequence, "forward")
    reversePrimer = screenPrimer(insert, "R")
    print(reversePrimer.sequence, "reverse")
    
    primers = addOverhangs(forwardPrimer, reversePrimer, "reverse", upstreamBackbone, downstreamBackbone)
    return (primers[0].sequence, primers[1].sequence)
def screenPrimer(insert, whichPrimer, primerSize=18): #Make primers for an insert to amplify it such that it can be assembled with Gibson
    if primerSize < 40:
        #Make sure you are selecting the right region to prime. 
        #This algorithm creates primers that amplify the WHOLE SEQUENCE minus the stop codon at the end.
        if whichPrimer == "F": 
            primer = DNASequence(insert.sequence[:primerSize])
        elif whichPrimer == "R":
            primer = DNASequence(reverseComplement(insert.sequence[len(insert.sequence) - primerSize: len(insert.sequence)]))
        else:
            print("error")
        #Add various screens below here. You can update the parameters to be more or less stringent depending on how likely you would like your primers to work
        isGood = True
        if primer.Tm < 58.0 or primer.Tm > 59.5:
            isGood = False
        if float(primer.Homodimer.dg) < -10000.0:
            isGood = False
        if float(primer.Hairpin.dg) < -5000.0:
            isGood = False
    
        #return the shortest (18bp ≤) desireable primers
        if isGood == True:
            #screening returned a good primer; return the primer
            return primer
        else:
            #primer failed screening, try again with a larger primer using recursion
            if whichPrimer == "F":
                primer = screenPrimer(insert, "F", primerSize + 1) 
            elif whichPrimer == "R":
                primer = screenPrimer(insert, "R", primerSize + 1)
            return primer
    else:
        #parameters are too stringent; no primers were found that matched the criteria given. Return an empty string
        print ("You cannot make primers with the given parameters")
        return DNASequence("")
def addOverhangs(primer1, primer2, orientation, leftBackbone, rightBackbone):
    #decide how the construct should be oriented in the final vector and add the overhangs so Gibson will add correctly.
    if orientation == "forward":
        primer2.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer2.sequence
        primer1.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer1.sequence
        return primer1, primer2
    elif orientation == "reverse":
        primer1.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer1.sequence
        primer2.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer2.sequence
        return primer1, primer2
    else:
        print ("please enter either a forward or reverse strand orientation")


        
insert = DNASequence('atgGAAAAGACATATAACCCACAAGATATCGAACAGCCGCTTTACGAGCACTGGGAAAAGCTGATTGAACAGCAGGCTGTTATCGCCGCGCTG'.upper())
leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())

rightbackbone = DNASequence('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgcca'.upper())

screenPrimers(insert, leftbackbone , rightbackbone)

ATGGAAAAGACATATAACCCACAAGATATCGAACA forward
CAGCGCGGCGATAACAGCC reverse


('CTAGAAATAATTTTGTTTAACTTTAAGAAGGAGATATACCATGGAAAAGACATATAACCCACAAGATATCGAACA',
 'CTTTGTTAGCAGCCGGATCTCAGTGGTGGTGGTGGTGGTGCAGCGCGGCGATAACAGCC')

In [238]:
reverseComplement('CGTTGAAGACCGTTTCAATGCGT'.upper())

'ACGCATTGAAACGGTCTTCAACG'

In [114]:
import primer3

primer3.calcTm('atggaaaagacatataacccacaagatatcgaac'.upper())

t = (reverseComplement('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgc'[:40].upper()))

print(t)

leftbackbone = ('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())

def reverse(sequence):
    reverse = ""
    for base in sequence:
        if base == "A":
            reverse = "A" + reverse 
        elif base == "T":
            reverse = "T" + reverse
        elif base == "G":
            reverse = "G" + reverse
        elif base == "C":
            reverse = "C" + reverse
        else:
            print ("error")
    return (reverse)

s = (leftbackbone)

s[len(s) - 40: len(s)]

CTAGAAATAATTTTGTTTAACTTTAAGAAGGAGATATACC


'CTTTGTTAGCAGCCGGATCTCAGTGGTGGTGGTGGTGGTG'

In [127]:
def long_substr(data):
    substr = ''
    if len(data) > 1 and len(data[0]) > 0:
        for i in range(len(data[0])):
            for j in range(len(data[0])-i+1):
                if j > len(substr) and is_substr(data[0][i:i+j], data):
                    substr = data[0][i:i+j]
    return substr

def is_substr(find, data):
    if len(data) < 1 and len(find) < 1:
        return False
    for i in range(len(data)):
        if find not in data[i]:
            return False
    return True


print (long_substr(['TCCCTATCAGTGATAGAGAAAAGAGAATCGGATCCAAAAGATCTAAAGAGGAGAAAGGATCTATGTTAATTGGCTATGTCCGTGTGTCCACCAACGATCAAAACACAGATCTTCAAAGAAATGCCCTCGTTTGCGCGGGATGTGAGCAGATCTTTGAAGATAAACTTAGCGGTACCCGTACCGATCGACCGGGTCTTAAACGCGCACTTAAACGGCTTCAGAAGGGGGATACCTTGGTCGTATGGAAACTGGACCGGCTGGGTAGATCGATGAAACATCTGATTAGTTTAGTTGGAGAACTGCGAGAACGCGGCATCAATTTTCGCTCCTTAACCGATAGCATCGATACCTCCAGTCCAATGGGGAGGTTTTTCTTCTATGTTATGGGCGCGCTGGCCGAAATGGAACGTGAATTGATTATCGAGCGGACAATGGCTGGGTTAGCGGCAGCACGAAATAAAGGACGGCGCTTCGGTCGTCCACCCAAGGGCGGCTCCGGTGGCTCGGGAGGTTCCGGCGGTAGCGGGGGCAGCGGCGGCAGTGGGGGCTCAGGAGGTTCTGATAAAAAATATTCTATAGGACTGGCTATTGGTACCAATTCAGTGGGTTGGGCGGTAATTACCGACGAATATAAAGTGCCGTCTAAAAAATTTAAGGTCTTGGGGAATACAGACCGTCACAGTATAAAAAAAAATCTGATCGGTGCGCTGCTGTTTGACTCTGGAGAAACTGCCGAAGCAACTCGTTTAAAACGAACCGCACGCCGTCGCTATACAAGGCGGAAAAATCGTATATGCTATTTACAAGAAATTTTTTCCAATGAAATGGCAAAAGTCGATGACAGCTTCTTTCATCGCTTAGAAGAGTCCTTTTTAGTTGAAGAAGATAAAAAACACGAACGCCATCCAATTTTTGGGAATATAGTTGATGAAGTCGCCTATCACGAAAAATATCCGACTATTTACCACCTGCGTAAGAAACTTGTGGATTCGACGGATAAAGCAGATCTGAGGCTGATTTATCTCGCCCTCGCACATATGATTAAATTCAGAGGTCATTTCCTGATAGAAGGCGATCTGAATCCCGACAACTCTGATGTTGATAAGTTGTTTATACAGCTGGTGCAAACCTATAATCAACTATTCGAAGAGAATCCTATAAACGCCAGCGGAGTGGACGCTAAAGCGATACTCAGCGCGCGCTTATCTAAAAGCCGGCGGCTGGAAAATCTGATCGCACAATTGCCAGGTGAAAAAAAAAATGGTTTATTTGGCAATCTGATAGCGCTTTCACTGGGGCTAACACCCAATTTTAAATCGAATTTTGATTTAGCCGAAGATGCTAAGCTGCAGCTGAGTAAAGATACCTATGATGACGATCTTGATAACCTGCTGGCACAAATTGGAGATCAATACGCAGATCTGTTTTTGGCCGCCAAGAATCTGTCGGATGCAATTCTGCTGAGCGATATTTTGAGGGTGAATACTGAAATCACTAAGGCACCGCTCTCTGCAAGTATGATCAAGCGTTATGATGAGCATCACCAGGACTTAACCCTGTTAAAAGCTCTGGTCCGCCAGCAGCTCCCGGAAAAATACAAGGAAATATTCTTCGATCAATCTAAGAATGGGTATGCCGGCTATATCGATGGCGGAGCGTCTCAGGAAGAATTTTATAAATTCATAAAACCTATCCTTGAAAAAATGGATGGAACGGAAGAGCTGCTGGTTAAACTGAATAGAGAGGATCTGTTACGCAAGCAGAGGACATTCGATAACGGTAGTATCCCACATCAAATACATCTGGGTGAACTTCACGCGATTCTTCGCCGCCAGGAAGACTTTTATCCCTTTCTAAAAGACAACAGGGAGAAAATTGAAAAAATCCTGACGTTTCGTATTCCTTATTATGTGGGGCCGCTGGCTCGTGGAAACAGTCGGTTCGCATGGATGACGCGCAAGTCCGAAGAGACGATAACGCCGTGGAATTTCGAAGAAGTGGTCGACAAAGGCGCAAGTGCACAGTCATTCATCGAACGTATGACTAACTTCGATAAAAATTTACCAAACGAGAAAGTTCTGCCTAAGCATAGCCTTCTCTATGAATACTTTACAGTCTATAACGAGTTAACTAAAGTTAAATATGTGACTGAAGGTATGCGTAAACCGGCGTTTCTGAGCGGCGAACAAAAAAAGGCAATTGTCGATCTGTTATTTAAAACAAATCGTAAAGTTACGGTCAAGCAATTGAAAGAGGACTACTTCAAAAAAATTGAGTGTTTCGACAGCGTCGAAATTTCTGGCGTTGAAGACCGTTTCAAT', 'CGTTGAAGACCGTTTCAATGCGTCGCTGGGAACTTACCACGATCTACTCAAAATCATAAAAGATAAAGACTTTCTGGACAATGAAGAGAACGAAGACATACTTGAAGACATCGTTCTGACCCTTACGCTATTTGAAGATAGGGAGATGATCGAAGAGCGCTTGAAGACCTATGCGCACCTATTCGATGATAAAGTTATGAAACAGCTCAAGCGGCGCCGTTACACGGGGTGGGGGCGTCTGTCGAGGAAGTTAATCAATGGAATTCGTGATAAGCAGTCAGGCAAGACGATCCTGGACTTTCTGAAATCAGATGGGTTTGCAAATCGTAACTTCATGCAGCTGATCCATGATGATTCGTTAACTTTTAAAGAAGATATTCAAAAAGCCCAAGTATCTGGTCAGGGTGACTCTCTTCACGAACATATCGCGAATTTGGCAGGTTCACCAGCTATTAAAAAAGGTATCTTGCAGACTGTGAAAGTTGTCGATGAACTTGTTAAAGTTATGGGTCGTCACAAGCCAGAGAATATCGTAATTGAGATGGCGCGTGAAAATCAAACAACCCAAAAAGGCCAGAAAAACTCGAGGGAACGCATGAAACGTATTGAAGAAGGTATTAAGGAGTTGGGTTCCCAGATACTTAAAGAACACCCGGTCGAAAATACGCAACTGCAAAATGAAAAATTATATCTATATTATCTTCAGAACGGGCGCGATATGTATGTGGACCAAGAGCTGGATATCAATCGGCTTTCGGATTATGACGTCGATGCAATCGTTCCGCAAAGTTTCCTCAAGGATGACAGCATCGATAACAAAGTGTTAACCCGTAGCGATAAAAACCGTGGCAAAAGTGATAACGTGCCATCGGAAGAGGTAGTGAAAAAGATGAAAAATTATTGGCGCCAGCTGCTTAATGCTAAGCTGATTACGCAACGGAAATTTGATAATCTGACTAAAGCTGAGCGGGGCGGGCTGTCGGAACTCGATAAAGCTGGGTTCATAAAAAGACAATTGGTCGAGACGCGCCAGATTACCAAACACGTAGCCCAAATTCTGGATTCACGGATGAATACGAAATACGACGAGAATGATAAACTGATCCGGGAAGTGAAAGTGATCACTCTTAAATCTAAACTGGTATCAGACTTTAGAAAAGATTTCCAATTCTATAAGGTTCGTGAAATCAACAATTATCATCACGCTCATGACGCCTATTTAAATGCGGTGGTAGGCACAGCGTTAATCAAGAAATATCCGAAACTTGAGTCAGAGTTTGTGTATGGAGATTATAAAGTTTATGATGTTAGAAAAATGATTGCTAAATCGGAGCAAGAAATTGGTAAGGCTACAGCGAAATATTTTTTTTACTCCAATATTATGAATTTCTTTAAGACTGAAATTACCTTGGCTAATGGTGAAATCCGCAAACGGCCGCTGATCGAAACAAATGGTGAAACTGGCGAGATTGTTTGGGATAAGGGCCGCGATTTCGCAACGGTCCGTAAGGTGTTGAGTATGCCTCAGGTGAATATTGTAAAGAAAACGGAAGTACAGACTGGCGGGTTTAGCAAAGAATCAATTTTGCCGAAACGTAATTCTGATAAACTGATCGCTCGTAAGAAAGATTGGGATCCCAAGAAGTATGGTGGCTTTGATAGTCCGACCGTAGCGTATAGTGTTCTGGTGGTAGCTAAGGTAGAAAAAGGCAAATCAAAGAAACTGAAGTCCGTGAAGGAACTACTGGGTATCACGATCATGGAACGATCAAGTTTTGAAAAAAATCCGATCGACTTCCTTGAAGCGAAAGGGTACAAAGAGGTAAAGAAGGACCTAATTATAAAACTGCCCAAATATAGTCTGTTTGAACTGGAAAATGGAAGGAAACGAATGTTGGCAAGCGCTGGTGAGCTGCAGAAAGGCAATGAATTAGCCCTGCCCTCGAAATATGTAAATTTCTTATACTTGGCCAGTCATTATGAAAAGCTGAAAGGATCACCTGAAGATAATGAACAAAAACAGCTCTTCGTCGAGCAACACAAGCATTACTTAGATGAAATTATAGAACAAATTTCTGAATTCTCAAAACGTGTTATATTAGCGGACGCGAATCTGGATAAGGTCTTGAGCGCATATAACAAGCATCGCGATAAACCGATACGGGAACAAGCGGAAAATATTATACATCTGTTCACGCTGACCAATTTAGGCGCGCCTGCTGCGTTTAAATATTTTGATACAACCATAGATCGTAAACGTTACACCAGCACAAAGGAAGTCCTGGATGCCACACTGATCCATCAGAGCATAACAGGACTGTACGAGACCCGTATCGATCTGAGTCAGCTTGGAGGTGATGGTGGTTCGGATTACAAGGATGACGATGATAAAGGTGGAAGTCCGAAAAAAAAAAGGAAAGTTTGATAACTCGAGTAAGGATCAGTAAGGATCTCCAGGCATC']))

#findHomology('TCCCTATCAGTGATAGAGAAAAGAGAATCGGATCCAAAAGATCTAAAGAGGAGAAAGGATCTATGTTAATTGGCTATGTCCGTGTGTCCACCAACGATCAAAACACAGATCTTCAAAGAAATGCCCTCGTTTGCGCGGGATGTGAGCAGATCTTTGAAGATAAACTTAGCGGTACCCGTACCGATCGACCGGGTCTTAAACGCGCACTTAAACGGCTTCAGAAGGGGGATACCTTGGTCGTATGGAAACTGGACCGGCTGGGTAGATCGATGAAACATCTGATTAGTTTAGTTGGAGAACTGCGAGAACGCGGCATCAATTTTCGCTCCTTAACCGATAGCATCGATACCTCCAGTCCAATGGGGAGGTTTTTCTTCTATGTTATGGGCGCGCTGGCCGAAATGGAACGTGAATTGATTATCGAGCGGACAATGGCTGGGTTAGCGGCAGCACGAAATAAAGGACGGCGCTTCGGTCGTCCACCCAAGGGCGGCTCCGGTGGCTCGGGAGGTTCCGGCGGTAGCGGGGGCAGCGGCGGCAGTGGGGGCTCAGGAGGTTCTGATAAAAAATATTCTATAGGACTGGCTATTGGTACCAATTCAGTGGGTTGGGCGGTAATTACCGACGAATATAAAGTGCCGTCTAAAAAATTTAAGGTCTTGGGGAATACAGACCGTCACAGTATAAAAAAAAATCTGATCGGTGCGCTGCTGTTTGACTCTGGAGAAACTGCCGAAGCAACTCGTTTAAAACGAACCGCACGCCGTCGCTATACAAGGCGGAAAAATCGTATATGCTATTTACAAGAAATTTTTTCCAATGAAATGGCAAAAGTCGATGACAGCTTCTTTCATCGCTTAGAAGAGTCCTTTTTAGTTGAAGAAGATAAAAAACACGAACGCCATCCAATTTTTGGGAATATAGTTGATGAAGTCGCCTATCACGAAAAATATCCGACTATTTACCACCTGCGTAAGAAACTTGTGGATTCGACGGATAAAGCAGATCTGAGGCTGATTTATCTCGCCCTCGCACATATGATTAAATTCAGAGGTCATTTCCTGATAGAAGGCGATCTGAATCCCGACAACTCTGATGTTGATAAGTTGTTTATACAGCTGGTGCAAACCTATAATCAACTATTCGAAGAGAATCCTATAAACGCCAGCGGAGTGGACGCTAAAGCGATACTCAGCGCGCGCTTATCTAAAAGCCGGCGGCTGGAAAATCTGATCGCACAATTGCCAGGTGAAAAAAAAAATGGTTTATTTGGCAATCTGATAGCGCTTTCACTGGGGCTAACACCCAATTTTAAATCGAATTTTGATTTAGCCGAAGATGCTAAGCTGCAGCTGAGTAAAGATACCTATGATGACGATCTTGATAACCTGCTGGCACAAATTGGAGATCAATACGCAGATCTGTTTTTGGCCGCCAAGAATCTGTCGGATGCAATTCTGCTGAGCGATATTTTGAGGGTGAATACTGAAATCACTAAGGCACCGCTCTCTGCAAGTATGATCAAGCGTTATGATGAGCATCACCAGGACTTAACCCTGTTAAAAGCTCTGGTCCGCCAGCAGCTCCCGGAAAAATACAAGGAAATATTCTTCGATCAATCTAAGAATGGGTATGCCGGCTATATCGATGGCGGAGCGTCTCAGGAAGAATTTTATAAATTCATAAAACCTATCCTTGAAAAAATGGATGGAACGGAAGAGCTGCTGGTTAAACTGAATAGAGAGGATCTGTTACGCAAGCAGAGGACATTCGATAACGGTAGTATCCCACATCAAATACATCTGGGTGAACTTCACGCGATTCTTCGCCGCCAGGAAGACTTTTATCCCTTTCTAAAAGACAACAGGGAGAAAATTGAAAAAATCCTGACGTTTCGTATTCCTTATTATGTGGGGCCGCTGGCTCGTGGAAACAGTCGGTTCGCATGGATGACGCGCAAGTCCGAAGAGACGATAACGCCGTGGAATTTCGAAGAAGTGGTCGACAAAGGCGCAAGTGCACAGTCATTCATCGAACGTATGACTAACTTCGATAAAAATTTACCAAACGAGAAAGTTCTGCCTAAGCATAGCCTTCTCTATGAATACTTTACAGTCTATAACGAGTTAACTAAAGTTAAATATGTGACTGAAGGTATGCGTAAACCGGCGTTTCTGAGCGGCGAACAAAAAAAGGCAATTGTCGATCTGTTATTTAAAACAAATCGTAAAGTTACGGTCAAGCAATTGAAAGAGGACTACTTCAAAAAAATTGAGTGTTTCGACAGCGTCGAAATTTCTGGCGTTGAAGACCGTTTCAAT', 'CGTTGAAGACCGTTTCAATGCGTCGCTGGGAACTTACCACGATCTACTCAAAATCATAAAAGATAAAGACTTTCTGGACAATGAAGAGAACGAAGACATACTTGAAGACATCGTTCTGACCCTTACGCTATTTGAAGATAGGGAGATGATCGAAGAGCGCTTGAAGACCTATGCGCACCTATTCGATGATAAAGTTATGAAACAGCTCAAGCGGCGCCGTTACACGGGGTGGGGGCGTCTGTCGAGGAAGTTAATCAATGGAATTCGTGATAAGCAGTCAGGCAAGACGATCCTGGACTTTCTGAAATCAGATGGGTTTGCAAATCGTAACTTCATGCAGCTGATCCATGATGATTCGTTAACTTTTAAAGAAGATATTCAAAAAGCCCAAGTATCTGGTCAGGGTGACTCTCTTCACGAACATATCGCGAATTTGGCAGGTTCACCAGCTATTAAAAAAGGTATCTTGCAGACTGTGAAAGTTGTCGATGAACTTGTTAAAGTTATGGGTCGTCACAAGCCAGAGAATATCGTAATTGAGATGGCGCGTGAAAATCAAACAACCCAAAAAGGCCAGAAAAACTCGAGGGAACGCATGAAACGTATTGAAGAAGGTATTAAGGAGTTGGGTTCCCAGATACTTAAAGAACACCCGGTCGAAAATACGCAACTGCAAAATGAAAAATTATATCTATATTATCTTCAGAACGGGCGCGATATGTATGTGGACCAAGAGCTGGATATCAATCGGCTTTCGGATTATGACGTCGATGCAATCGTTCCGCAAAGTTTCCTCAAGGATGACAGCATCGATAACAAAGTGTTAACCCGTAGCGATAAAAACCGTGGCAAAAGTGATAACGTGCCATCGGAAGAGGTAGTGAAAAAGATGAAAAATTATTGGCGCCAGCTGCTTAATGCTAAGCTGATTACGCAACGGAAATTTGATAATCTGACTAAAGCTGAGCGGGGCGGGCTGTCGGAACTCGATAAAGCTGGGTTCATAAAAAGACAATTGGTCGAGACGCGCCAGATTACCAAACACGTAGCCCAAATTCTGGATTCACGGATGAATACGAAATACGACGAGAATGATAAACTGATCCGGGAAGTGAAAGTGATCACTCTTAAATCTAAACTGGTATCAGACTTTAGAAAAGATTTCCAATTCTATAAGGTTCGTGAAATCAACAATTATCATCACGCTCATGACGCCTATTTAAATGCGGTGGTAGGCACAGCGTTAATCAAGAAATATCCGAAACTTGAGTCAGAGTTTGTGTATGGAGATTATAAAGTTTATGATGTTAGAAAAATGATTGCTAAATCGGAGCAAGAAATTGGTAAGGCTACAGCGAAATATTTTTTTTACTCCAATATTATGAATTTCTTTAAGACTGAAATTACCTTGGCTAATGGTGAAATCCGCAAACGGCCGCTGATCGAAACAAATGGTGAAACTGGCGAGATTGTTTGGGATAAGGGCCGCGATTTCGCAACGGTCCGTAAGGTGTTGAGTATGCCTCAGGTGAATATTGTAAAGAAAACGGAAGTACAGACTGGCGGGTTTAGCAAAGAATCAATTTTGCCGAAACGTAATTCTGATAAACTGATCGCTCGTAAGAAAGATTGGGATCCCAAGAAGTATGGTGGCTTTGATAGTCCGACCGTAGCGTATAGTGTTCTGGTGGTAGCTAAGGTAGAAAAAGGCAAATCAAAGAAACTGAAGTCCGTGAAGGAACTACTGGGTATCACGATCATGGAACGATCAAGTTTTGAAAAAAATCCGATCGACTTCCTTGAAGCGAAAGGGTACAAAGAGGTAAAGAAGGACCTAATTATAAAACTGCCCAAATATAGTCTGTTTGAACTGGAAAATGGAAGGAAACGAATGTTGGCAAGCGCTGGTGAGCTGCAGAAAGGCAATGAATTAGCCCTGCCCTCGAAATATGTAAATTTCTTATACTTGGCCAGTCATTATGAAAAGCTGAAAGGATCACCTGAAGATAATGAACAAAAACAGCTCTTCGTCGAGCAACACAAGCATTACTTAGATGAAATTATAGAACAAATTTCTGAATTCTCAAAACGTGTTATATTAGCGGACGCGAATCTGGATAAGGTCTTGAGCGCATATAACAAGCATCGCGATAAACCGATACGGGAACAAGCGGAAAATATTATACATCTGTTCACGCTGACCAATTTAGGCGCGCCTGCTGCGTTTAAATATTTTGATACAACCATAGATCGTAAACGTTACACCAGCACAAAGGAAGTCCTGGATGCCACACTGATCCATCAGAGCATAACAGGACTGTACGAGACCCGTATCGATCTGAGTCAGCTTGGAGGTGATGGTGGTTCGGATTACAAGGATGACGATGATAAAGGTGGAAGTCCGAAAAAAAAAAGGAAAGTTTGATAACTCGAGTAAGGATCAGTAAGGATCTCCAGGCATC')

CGTTGAAGACCGTTTCAAT


In [239]:
import primer3
import pandas
import math
sequences = []
breakdown = []
data = pandas.read_excel("/Users/aleclourenco/Desktop/REU-2017-Endy-Lab/PURE Datasheet.xlsx", sheetname="List of PURE Components", skiprows=[0])

class DNASequence():
    def __init__(self, sequence):
        #Initialize the DNA variable, which has sequence info as well as additional properties (Tm, homodimer & hairpin formation)
        self.sequence = sequence
        self.composition = self.getComposition(sequence)
        if len(sequence) != 0:
            self.gcContent = self.getGCcontent(sequence)
        else:
            self.gcContent = 0
        if len(sequence) < 60:
            self.Tm = primer3.calcTm(sequence)
            self.Homodimer = primer3.calcHomodimer(sequence)
            self.Hairpin = primer3.calcHairpin(sequence)
        else:
            self.Tm = 100000 #set the Tm arbitrarily high
        
    def getComposition(self, sequence):
        composition = {"A" : 0, "G" : 0, "C" : 0, "T" : 0}
        for base in sequence:
            if base == "A":
                composition["A"] = composition["A"] + 1
            elif base == "T":
                composition["T"] = composition["T"] + 1
            elif base == "G":
                composition["G"] = composition["G"] + 1
            elif base == "C":
                composition["C"] = composition["C"] + 1
            else:
                print("error 2")
        return composition
    def getGCcontent(self, sequence):
        GCpercent = (self.composition["G"] + self.composition["C"])/(self.composition["G"] + self.composition["T"] + self.composition["A"] + self.composition["C"])
        return GCpercent
    def reverseComplement(self, sequence):
        #Takes the complement of the sequence and reverses it.
        reverseComplement = ""
        for base in sequence:
            if base == "A":
                reverseComplement = "T" + reverseComplement
            elif base == "T":
                reverseComplement = "A" + reverseComplement
            elif base == "G":
                reverseComplement = "C" + reverseComplement
            elif base == "C":
                reverseComplement = "G" + reverseComplement 
            else:
                print ("error")
        return (reverseComplement)
def reverseComplement(sequence):
    #Takes the complement of the sequence and reverses it.
    reverseComplement = ""
    for base in sequence:
        if base == "A":
            reverseComplement = "T" + reverseComplement 
        elif base == "T":
            reverseComplement = "A" + reverseComplement
        elif base == "G":
            reverseComplement = "C" + reverseComplement
        elif base == "C":
            reverseComplement = "G" + reverseComplement
        else:
            print ("error")
    return (reverseComplement)
def screenPrimers(insert, upstreamBackbone, downstreamBackbone):
    #run the main program to create and screen the forward and reverse primers and then add overhangs.
    forwardPrimer = screenPrimer(insert, "F")
    #print(forwardPrimer.sequence, "forward")
    reversePrimer = screenPrimer(insert, "R")
    #print(reversePrimer.sequence, "reverse")
    if len(forwardPrimer.sequence) and len(reversePrimer.sequence) != 0:
        primers = addOverhangs(forwardPrimer, reversePrimer, "reverse", upstreamBackbone, downstreamBackbone)
        return (primers[0].sequence, primers[1].sequence)
    else:
        return ("You cannot make primers with the given parameters", "You cannot make primers with the given parameters")
    
def screenPrimer(insert, whichPrimer, primerSize=18): #Make primers for an insert to amplify it such that it can be assembled with Gibson
    if primerSize < 40:
        #Make sure you are selecting the right region to prime. 
        #This algorithm creates primers that amplify the WHOLE SEQUENCE minus the stop codon at the end.
        if whichPrimer == "F": 
            primer = DNASequence(insert.sequence[:primerSize])
        elif whichPrimer == "R":
            primer = DNASequence(reverseComplement(insert.sequence[len(insert.sequence) - primerSize: len(insert.sequence)]))
        else:
            print("error")
        #Add various screens below here. You can update the parameters to be more or less stringent depending on how likely you would like your primers to work
        isGood = True
        if primer.Tm < 58.0 or primer.Tm > 59.5:
            isGood = False
        if float(primer.Homodimer.dg) < -10000.0:
            isGood = False
        if float(primer.Hairpin.dg) < -5000.0:
            isGood = False
    
        #return the shortest (18bp ≤) desireable primers
        if isGood == True:
            #screening returned a good primer; return the primer
            return primer
        else:
            #primer failed screening, try again with a larger primer using recursion
            if whichPrimer == "F":
                primer = screenPrimer(insert, "F", primerSize + 1) 
            elif whichPrimer == "R":
                primer = screenPrimer(insert, "R", primerSize + 1)
            return primer
    else:
        #parameters are too stringent; no primers were found that matched the criteria given. Return an empty string
        return DNASequence("")
def addOverhangs(primer1, primer2, orientation, leftBackbone, rightBackbone):
    #decide how the construct should be oriented in the final vector and add the overhangs so Gibson will add correctly.
    if orientation == "forward":
        primer2.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer2.sequence
        primer1.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer1.sequence
        return primer1, primer2
    elif orientation == "reverse":
        primer1.sequence = reverseComplement(rightbackbone.sequence[:40]) + primer1.sequence
        primer2.sequence = leftBackbone.sequence[len(leftBackbone.sequence) - 40: len(leftBackbone.sequence)] + primer2.sequence
        return primer1, primer2
    else:
        print ("please enter either a forward or reverse strand orientation")
def formatSeq(oldSeq):
    if not type(oldSeq) is str:
        oldSeq = ""
    new = oldSeq.replace("\n", "")
    for number in range(0,10):
        temp = new.replace(str(number), "")
        new = temp
    oldSeq = new
    new = oldSeq.replace(" ", "")
    return new.upper()

def appendPrimers(data, insert, tag, primerList, nameList):
    name = str(data.ix[number,"Protein Name"]).replace(" ", "") 
    nameFOR = name + " FOR " + tag + "-tagged"
    nameREV = name + " REV " + tag + "-tagged"
    primerList.append(screenPrimers(insert, leftbackbone , rightbackbone)[0])
    nameList.append(nameFOR)
    primerList.append(screenPrimers(insert, leftbackbone , rightbackbone)[1])
    nameList.append(nameREV)


primerList = []
nameList = []

for number in range(0,49):
    tag = ""
    if (data.ix[number, "His Tagged Where?"]) == "C":
        tag = "C"
        #Correct the backbone so that the primers generated add in the correct area of the backbone
        leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())
        rightbackbone = DNASequence('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgcca'.upper())
        insert = DNASequence(formatSeq(data.ix[number,"Nucleotide Sequence"]))
        if len(insert.sequence) != 0:
            appendPrimers(data, insert, tag, primerList, nameList)
    elif (data.ix[number, "His Tagged Where?"]) == "N": 
        tag = "N"
        #Correct the backbone so that the primers generated add in the correct area of the backbone
        leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatc'.upper())
        rightbackbone = DNASequence('atggctgccgcgcggcaccaggccgctgctgtgatgatgatgatgatggctgctgcccatggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccag'.upper())
        insert = DNASequence(formatSeq(data.ix[number,"Nucleotide Sequence"]))
        if len(insert.sequence) != 0:
            appendPrimers(data, insert, tag, primerList, nameList)
    else: #We don't know whether unspecified proteins are N or C His-tagged, so we assume they could be either.
        leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatctcagtggtggtggtggtggtg'.upper())
        rightbackbone = DNASequence('ggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgccattgctgtggaagctgcctgcactaatgttccggcgttatttcttgatgtctctgaccagacacccatcaacagtattattttctcccatgaagacggtacgcgactgggcgtggagcatctggtcgcattgggtcaccagcaaatcgcgctgttagcgggcccattaagttctgtctcggcgcgtctgcgtctggctggctggcataaatatctcactcgcaatcaaattcagccgatagcggaacgggaaggcgactggagtgccatgtccggttttcaacaaaccatgcaaatgctgaatgagggcatcgttcccactgcgatgctggttgccaacgatcagatggcgctgggcgcaatgcgcgccattaccgagtccgggctgcgcgttggtgcggatatctcggtagtgggatacgacgataccgaagacagctcatgttatatcccgccgttaaccaccatcaaacaggattttcgcctgctggggcaaaccagcgtggaccgcttgctgcaactctctcagggccaggcggtgaagggcaatcagctgttgcccgtctcactggtgaaaagaaaaaccaccctggcgcccaatacgcaaaccgcctctccccgcgcgttggccgattcattaatgcagctggcacgacaggtttcccgactggaaagcgggcagtgagcgcaacgcaattaatgtaagttagctcactcattaggcaccgggatctcgaccgatgcccttgagagccttcaacccagtcagctccttccggtgggcgcggggcatgactatcgtcgccgcacttatgactgtcttctttatcatgcaactcgtaggacaggtgccggcagcgctctgggtcattttcggcgaggaccgctttcgctggagcgcgacgatgatcggcctgtcgcttgcggtattcggaatcttgcacgccctcgctcaagccttcgtcactggtcccgccaccaaacgtttcggcgagaagcaggccattatcgccggcatggcggccccacgggtgcgcatgatcgtgctcctgtcgttgaggacccggctaggctggcggggttgccttactggttagcagaatgaatcaccgatacgcgagcgaacgtgaagcgactgctgctgcaaaacgtctgcgacctgagcaacaacatgaatggtcttcggtttccgtgtttcgtaaagtctggaaacgcggaagtcagcgccctgcaccattatgttccggatctgcatcgcaggatgctgctggctaccctgtggaacacctacatctgtattaacgaagcgctggcattgaccctgagtgatttttctctggtcccgccgcatccataccgccagttgtttaccctcacaacgttccagtaaccgggcatgttcatcatcagtaacccgtatcgtgagcatcctctctcgtttcatcggtatcattacccccatgaacagaaatcccccttacacggaggcatcagtgaccaaacaggaaaaaaccgcccttaacatggcccgctttatcagaagccagacattaacgcttctggagaaactcaacgagctggacgcggatgaacaggcagacatctgtgaatcgcttcacgaccacgctgatgagctttaccgcagctgcctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcgcgtcagcgggtgttggcgggtgtcggggcgcagccatgacccagtcacgtagcgatagcggagtgtatactggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgaacaataaaactgtctgcttacataaacagtaatacaaggggtgttatgagccatattcaacgggaaacgtcttgctctaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccgggaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataaacttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaagaattaattcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgaaattgtaaacgttaatattttgttaaaattcgcgttaaatttttgttaaatcagctcattttttaaccaataggccgaaatcggcaaaatcccttataaatcaaaagaatagaccgagatagggttgagtgttgttccagtttggaacaagagtccactattaaagaacgtggactccaacgtcaaagggcgaaaaaccgtctatcagggcgatggcccactacgtgaaccatcaccctaatcaagttttttggggtcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctgcgcgtaaccaccacacccgccgcgcttaatgcgccgctacagggcgcgtcccattcgcca'.upper())
        insert = DNASequence(formatSeq(data.ix[number,"Nucleotide Sequence"]))
        if len(insert.sequence) != 0:
            tag = "C"
            appendPrimers(data, insert, tag, primerList, nameList)
        leftbackbone = DNASequence('atccggatatagttcctcctttcagcaaaaaacccctcaagacccgtttagaggccccaaggggttatgctagttattgctcagcggtggcagcagccaactcagcttcctttcgggctttgttagcagccggatc'.upper())
        rightbackbone = DNASequence('atggctgccgcgcggcaccaggccgctgctgtgatgatgatgatgatggctgctgcccatggtatatctccttcttaaagttaaacaaaattatttctagaggggaattgttatccgctcacaattcccctatagtgagtcgtattaatttcgcgggatcgagatctcgatcctctacgccggacgcatcgtggccggcatcaccggcgccacaggtgcggttgctggcgcctatatcgccgacatcaccgatggggaagatcgggctcgccacttcgggctcatgagcgcttgtttcggcgtgggtatggtggcaggccccgtggccgggggactgttgggcgccatctccttgcatgcaccattccttgcggcggcggtgctcaacggcctcaacctactactgggctgcttcctaatgcaggagtcgcataagggagagcgtcgagatcccggacaccatcgaatggcgcaaaacctttcgcggtatggcatgatagcgcccggaagagagtcaattcagggtggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccag'.upper())
        if len(insert.sequence) != 0:
            tag = "N"
            appendPrimers(data, insert, tag, primerList, nameList)
        
dataframe = pandas.DataFrame(primerList, nameList)

writer = pandas.ExcelWriter('test2.xlsx', engine='openpyxl')

dataframe.to_excel(writer)

writer.save()

In [165]:
data.ix[1,'Nucleotide Sequence']

'atgGCCAAAG AAGACAATAT TGAAATGCAA GGTACCGTTC TTGAAACGTT GCCTAATACC\nATGTTCCGCG TAGAGTTAGA AAACGGTCAC GTGGTTACTG CACACATCTC CGGTAAAATG\nCGCAAAAACT ACATCCGCAT CCTGACGGGC GACAAAGTGA CTGTTGAACT GACCCCGTAC\nGACCTGAGCA AAGGCCGCAT TGTCTTCCGT AGTCGCtga'

In [166]:
data.columns

Index(['Abbreviation', 'Gene Name', 'Protein Name', 'Function', 'Reference',
       'Sequence', 'Nucleotide Sequence', 'Amount Added (ug)', 'AA Sequence',
       'Methionine', 'Alanine', 'Arginine', 'Asparagine', 'Aspartic Acid',
       'Cysteine', 'Glutamic Acid', 'Glutamine', 'Glycine', 'Histidine',
       'Isoleucine', 'Leucine', 'Lysine', 'Phenylalanine', 'Proline', 'Serine',
       'Threonine', 'Tryptophan', 'Tyrosine', 'Valine'],
      dtype='object')

In [163]:
data.head()

,Abbreviation,Gene Name,Protein Name,Function,Reference,Sequence,Nucleotide Sequence,Amount Added (ug),AA Sequence,Methionine,...,Isoleucine,Leucine,Lysine,Phenylalanine,Proline,Serine,Threonine,Tryptophan,Tyrosine,Valine
0,Transcription/Translation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IF1,infA,Initiation Factor 1,Not fully understood but it is suggested that ...,Grunberg-Manago et al. 2014,NP_415404,atgGCCAAAG AAGACAATAT TGAAATGCAA GGTACCGTTC TT...,1,MAKEDNIEMQ GTVLETLPNT MFRVELENGH VVTAHISGKM RK...,4.0,...,5.0,6.0,5.0,2.0,2.0,3.0,7.0,0.0,2.0,7.0
2,IF2,infB,Initiation Factor 2,IF2 is implicated in fMet-tRNA_fMet binding to...,Grunberg-Manago et al. 2014,NP_417637,atgACAGATG TAACGATTAA AACGCTGGCC GCAGAGCGAC AG...,2,MTDVTIKTLA AERQTSVERL VQQFADAGIR KSADDSVSAQ EK...,21.0,...,47.0,54.0,66.0,15.0,21.0,44.0,42.0,2.0,11.0,83.0
3,IF3,infC,Initiation Factor 3,IF3 discriminates against noninitiator tRNAs b...,Grunberg-Manago et al. 2014,NP_416233,attAAAGGCG GAAAACGAGT TCAAACGGCG CGCCCTAACC GT...,0.75,MKGGKRVQTA RPNRINGEIR AQEVRLTGLE GEQLGIVSLR EA...,6.0,...,12.0,15.0,21.0,5.0,7.0,7.0,5.0,0.0,3.0,14.0
4,EF-G,fusA,Elongation Factor G,Facilitates the translocation of the ribosome ...,Chen et al. 2016,NP_417799,atgGCTCGTA CAACACCCAT CGCACGCTAC CGTAACATCG GT...,1,MARTTPIARY RNIGISAHID AGKTTTTERI LFYTGVNHKI GE...,23.0,...,50.0,43.0,44.0,24.0,33.0,29.0,38.0,6.0,19.0,60.0


In [193]:
import datetime as dt
import pandas as pd
from openpyxl import load_workbook

def add_frame_to_workbook(filename, tabname, dataframe, timestamp):
    """
    Save a dataframe to a workbook tab with the filename and tabname
    coded to timestamp

    :param filename: filename to create, can use strptime formatting
    :param tabname: tabname to create, can use strptime formatting
    :param dataframe: dataframe to save to workbook
    :param timestamp: timestamp associated with dataframe
    :return: None
    """
    filename = timestamp.strftime(filename)
    sheet_name = timestamp.strftime(tabname)

    # create a writer for this month and year
    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # copy existing sheets
        writer.sheets = dict(
            (ws.title, ws) for ws in writer.book.worksheets)
    except IOError:
        # file does not exist yet, we will create it
        pass

    # write out the new sheet
    dataframe.to_excel(writer, sheet_name=sheet_name)

    # save the workbook
    writer.save()

data = [x.strip().split() for x in """
            Primer Name  Sequence
                    FOR  ATGTGCCATGA...
                    REV  TGCCTAGTCA...
""".split('\n')[1:-1]]
df = pd.DataFrame(data=data[1:], columns=data[0])

name_template = './sample-%m-%y.xlsx'
tab_template = '%d_%H_%M'
now = dt.datetime.now()
in_an_hour = now + dt.timedelta(hours=1)
add_frame_to_workbook(name_template, tab_template, df, now)
add_frame_to_workbook(name_template, tab_template, df, in_an_hour)

In [223]:
import pandas as pd
import numpy as np


df = pd.DataFrame({'name' : pd.Series(np.random.randn(3)),'Sequence' : pd.Series(np.random.randn(4))})
print(df)

writer = pd.ExcelWriter('test2.xlsx', engine='openpyxl')

df.to_excel(writer)

writer.save()

   Sequence      name
0 -0.420077  0.081306
1  0.578674  2.456459
2 -0.355857  0.524907
3 -0.450266       NaN
